In [ ]:
#compute optimal thershold sex

In [1]:
import os
import sys
import json
import copy
import socket
import getpass
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
sys.path.append('../')
import pickle
import warnings
from sklearn.metrics import recall_score, matthews_corrcoef, roc_auc_score, f1_score
from collections import defaultdict
import json

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def flatten(l):
    return [item for sublist in l for item in sublist]

def compute_opt_thres(target, pred):
    opt_thres = 0
    opt_f1 = 0
    for i in np.arange(0.001, 0.999, 0.001):
        f1 = f1_score(target, pred >= i)
        if f1 >= opt_f1:
            opt_thres = i
            opt_f1 = f1
    return opt_thres

plt.rcParams.update({'font.size': 20})

In [2]:
#root_dir = Path(f'/path/to/your/root')
root_dir = Path(f'/home/lchanch/initial_training/output_sweep_12/grid_sex_mimic_12')

In [3]:
if Path('opt_thres.pkl').is_file():
    already_computed = set(pickle.load(Path('opt_thres.pkl').open('rb')).keys())
else:
    already_computed = set()

In [4]:
results = []
for i in tqdm(root_dir.glob('**/done')):
    args = json.load((i.parent/'args.json').open('r'))
    if (args['dataset'][0], args['task'], args['attr'], args['algorithm']) in already_computed:
        continue
    
    final_res = pickle.load((i.parent/'final_results.pkl').open('rb'))
    
    ssets = ['va', 'te', 'MIMIC-sex-te', 'CheXpert-sex-te']
  
    for sset in ssets:
        if sset in final_res:
            args[f'{sset}_auroc'] = final_res[sset]['overall']['AUROC']
            if sset == 'va':
                args[f'{sset}_min_attr_auroc'] = final_res[sset]['min_attr']['AUROC']
    args['va_y'] = final_res['va']['y']
    args['va_preds'] = final_res['va']['preds']
    
    results.append(args)
df = pd.DataFrame(results)

136it [00:00, 591.10it/s]


In [5]:
df['dataset'] = df['dataset'].apply(lambda x: x[0])

In [8]:
df.shape

(136, 33)

In [7]:
df.head()

,store_name,dataset,task,attr,group_def,algorithm,output_dir,data_dir,hparams,hparams_seed,...,checkpoint_freq,skip_model_save,debug,image_arch,aug,va_auroc,va_min_attr_auroc,te_auroc,va_y,va_preds
0,d625628b73ed041fe6e242b79f4575e3,MIMIC,Cardiomegaly,sex,group,ERM,/home/lchanch/initial_training/output_sweep_12...,/home/lchanch/initial_training/image_df,{},7,...,1000,False,False,densenet_sup_in1k,basic2,0.805573,0.799530,0.801891,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.10041881, 0.098386146, 0.10390391, 0.100681..."
1,5b13d76ffe1c4e5727a28d8dc8cf4ad7,MIMIC,Cardiomegaly,sex,group,ERM,/home/lchanch/initial_training/output_sweep_12...,/home/lchanch/initial_training/image_df,{},11,...,1000,False,False,densenet_sup_in1k,basic2,0.520238,0.516330,0.521601,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7446e1af1146a1fe82ffb966c98540d5,MIMIC,Cardiomegaly,sex,group,ERM,/home/lchanch/initial_training/output_sweep_12...,/home/lchanch/initial_training/image_df,{},4,...,1000,False,False,densenet_sup_in1k,basic2,0.566794,0.566446,0.562549,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ..."
3,b781cd45b64b83c4329776e029748840,MIMIC,Pneumothorax,sex,group,ERM,/home/lchanch/initial_training/output_sweep_12...,/home/lchanch/initial_training/image_df,{},11,...,1000,False,False,densenet_sup_in1k,basic2,0.508278,0.507261,0.502025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0bdd44bdd53fb5f084305a8aa360675e,MIMIC,Pleural Effusion,sex,group,ERM,/home/lchanch/initial_training/output_sweep_12...,/home/lchanch/initial_training/image_df,{},11,...,1000,False,False,densenet_sup_in1k,basic2,0.500000,0.500000,0.500000,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
#optimal_thresholds

In [ ]:
#best models según va_min_attr_auroc

In [9]:
best_models = df.groupby(['dataset', 'task', 'attr', 'algorithm']).apply(lambda x: x.loc[x['va_min_attr_auroc'].idxmax()])

/tmp/ipykernel_3786778/3613221529.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_models = df.groupby(['dataset', 'task', 'attr', 'algorithm']).apply(lambda x: x.loc[x['va_min_attr_auroc'].idxmax()])


In [10]:
best_models

store_name  \
dataset task             attr algorithm                                     
MIMIC   Cardiomegaly     sex  ERM        62da7a4cc63a8e6972347e6949106c6e   
        No Finding       sex  ERM        8567bca8c4876916928b7465a52cdeae   
        Pleural Effusion sex  ERM        b73aa9e925863665502c5e22300e663f   
        Pneumothorax     sex  ERM        2254ef4aeff39e3bbd73d8f30e0567a1   

                                        dataset              task attr  \
dataset task             attr algorithm                                  
MIMIC   Cardiomegaly     sex  ERM         MIMIC      Cardiomegaly  sex   
        No Finding       sex  ERM         MIMIC        No Finding  sex   
        Pleural Effusion sex  ERM         MIMIC  Pleural Effusion  sex   
        Pneumothorax     sex  ERM         MIMIC      Pneumothorax  sex   

                                        group_def algorithm  \
dataset task             attr algorithm                       
MIMIC   Cardiomegaly     sex  ERM           group       ERM   
        No Finding       sex  ERM           group       ERM   
        Pleural Effusion sex  ERM           group       ERM   
        Pneumothorax     sex  ERM           group       ERM   

                                                                                output_dir  \
dataset task             attr algorithm                                                      
MIMIC   Cardiomegaly     sex  ERM        /home/lchanch/initial_training/output_sweep_12...   
        No Finding       sex  ERM        /home/lchanch/initial_training/output_sweep_12...   
        Pleural Effusion sex  ERM        /home/lchanch/initial_training/output_sweep_12...   
        Pneumothorax     sex  ERM        /home/lchanch/initial_training/output_sweep_12...   

                                                                        data_dir  \
dataset task             attr algorithm                                            
MIMIC   Cardiomegaly     sex  ERM        /home/lchanch/initial_training/image_df   
        No Finding       sex  ERM        /home/lchanch/initial_training/image_df   
        Pleural Effusion sex  ERM        /home/lchanch/initial_training/image_df   
        Pneumothorax     sex  ERM        /home/lchanch/initial_training/image_df   

                                        hparams  hparams_seed  ...  \
dataset task             attr algorithm                        ...   
MIMIC   Cardiomegaly     sex  ERM            {}             6  ...   
        No Finding       sex  ERM            {}             1  ...   
        Pleural Effusion sex  ERM            {}             1  ...   
        Pneumothorax     sex  ERM            {}             1  ...   

                                         checkpoint_freq skip_model_save  \
dataset task             attr algorithm                                    
MIMIC   Cardiomegaly     sex  ERM                   1000           False   
        No Finding       sex  ERM                   1000           False   
        Pleural Effusion sex  ERM                   1000           False   
        Pneumothorax     sex  ERM                   1000           False   

                                         debug         image_arch     aug  \
dataset task             attr algorithm                                     
MIMIC   Cardiomegaly     sex  ERM        False  densenet_sup_in1k  basic2   
        No Finding       sex  ERM        False  densenet_sup_in1k  basic2   
        Pleural Effusion sex  ERM        False  densenet_sup_in1k  basic2   
        Pneumothorax     sex  ERM        False  densenet_sup_in1k  basic2   

                                         va_auroc va_min_attr_auroc  te_auroc  \
dataset task             attr algorithm                                         
MIMIC   Cardiomegaly     sex  ERM        0.815654          0.812019  0.810155   
        No Finding       sex  ERM        0.846209          0.838359  0.849310   
        Pleural Effusion sex  ERM        

In [11]:
opt_thres = {}
for idx, row in tqdm(best_models.iterrows(), total = len(best_models)):
    dataset, task, attr, algorithm = idx
#     if dataset not in opt_thres:
#         opt_thres[dataset] = {}
    opt_thres[(dataset, task, attr, algorithm)] = np.round(compute_opt_thres(row['va_y'], row['va_preds']), 3)

100%|██████████| 4/4 [00:21<00:00,  5.28s/it]


In [12]:
if Path('opt_thres.pkl').is_file():
    old_file = pickle.load(Path('opt_thres.pkl').open('rb'))
else:
    old_file = {}

In [13]:
opt_thres = {**old_file, **opt_thres}

In [14]:
opt_thres

{('MIMIC', 'Cardiomegaly', 'sex', 'ERM'): 0.2,
 ('MIMIC', 'No Finding', 'sex', 'ERM'): 0.425,
 ('MIMIC', 'Pleural Effusion', 'sex', 'ERM'): 0.345,
 ('MIMIC', 'Pneumothorax', 'sex', 'ERM'): 0.132}

In [ ]:
#se observa que los opt thersholds son los mismos excepto para cardiomegaly, que en el de 3 semillas era 0.245

In [16]:
pickle.dump(opt_thres, open('opt_thres_sex_mimic_12.pkl', 'wb'))

In [17]:
best_models[['store_name','hparams_seed']]

store_name  \
dataset task             attr algorithm                                     
MIMIC   Cardiomegaly     sex  ERM        62da7a4cc63a8e6972347e6949106c6e   
        No Finding       sex  ERM        8567bca8c4876916928b7465a52cdeae   
        Pleural Effusion sex  ERM        b73aa9e925863665502c5e22300e663f   
        Pneumothorax     sex  ERM        2254ef4aeff39e3bbd73d8f30e0567a1   

                                         hparams_seed  
dataset task             attr algorithm                
MIMIC   Cardiomegaly     sex  ERM                   6  
        No Finding       sex  ERM                   1  
        Pleural Effusion sex  ERM                   1  
        Pneumothorax     sex  ERM                   1